In [2]:
import pandas as pd

In [3]:
data = pd.read_csv("dataset/housing.csv")

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 545 entries, 0 to 544
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   price             545 non-null    int64 
 1   area              545 non-null    int64 
 2   bedrooms          545 non-null    int64 
 3   bathrooms         545 non-null    int64 
 4   stories           545 non-null    int64 
 5   mainroad          545 non-null    object
 6   guestroom         545 non-null    object
 7   basement          545 non-null    object
 8   hotwaterheating   545 non-null    object
 9   airconditioning   545 non-null    object
 10  parking           545 non-null    int64 
 11  prefarea          545 non-null    object
 12  furnishingstatus  545 non-null    object
dtypes: int64(6), object(7)
memory usage: 55.5+ KB


In [5]:
data.head(5)

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished


In [9]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline

# Separate target variable and features
target = data['price']
features = data.drop('price', axis=1)

# Identifying categorical and numeric columns
categorical_columns = ['mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'prefarea', 'furnishingstatus']
numeric_columns = ['area', 'bedrooms', 'bathrooms', 'stories', 'parking']

# Create transformers for numeric and categorical data
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),  # Filling missing values if any
    ('scaler', StandardScaler())  # Standardizing data
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),  # Filling missing values if any
    ('onehot', OneHotEncoder(handle_unknown='ignore', drop='first'))  # One hot encoding, dropping first to avoid multicollinearity
])

# Combine transformers into a preprocessor with ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_columns),
        ('cat', categorical_transformer, categorical_columns)
    ])

# Applying the preprocessor
features_preprocessed = preprocessor.fit_transform(features)

# Converting the transformed data back into a DataFrame
# Extracting new column names from the one-hot encoder
cat_features = preprocessor.named_transformers_['cat']['onehot'].get_feature_names_out()
new_columns = list(numeric_columns) + list(cat_features)

# Create the DataFrame
cleaned_data = pd.DataFrame(features_preprocessed, index=data.index, columns=new_columns)

# Display the first few rows to verify
print(cleaned_data.head())


       area  bedrooms  bathrooms   stories   parking  x0_yes  x1_yes  x2_yes  \
0  1.046726  1.403419   1.421812  1.378217  1.517692     1.0     0.0     0.0   
1  1.757010  1.403419   5.405809  2.532024  2.679409     1.0     0.0     0.0   
2  2.218232  0.047278   1.421812  0.224410  1.517692     1.0     0.0     1.0   
3  1.083624  1.403419   1.421812  0.224410  2.679409     1.0     0.0     1.0   
4  1.046726  1.403419  -0.570187  0.224410  1.517692     1.0     1.0     1.0   

   x3_yes  x4_yes  x5_yes  x6_semi-furnished  x6_unfurnished  
0     0.0     1.0     1.0                0.0             0.0  
1     0.0     1.0     0.0                0.0             0.0  
2     0.0     0.0     1.0                1.0             0.0  
3     0.0     1.0     1.0                0.0             0.0  
4     0.0     1.0     0.0                0.0             0.0  


In [10]:
cleaned_data

,area,bedrooms,bathrooms,stories,parking,x0_yes,x1_yes,x2_yes,x3_yes,x4_yes,x5_yes,x6_semi-furnished,x6_unfurnished
0,1.046726,1.403419,1.421812,1.378217,1.517692,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
1,1.757010,1.403419,5.405809,2.532024,2.679409,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,2.218232,0.047278,1.421812,0.224410,1.517692,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
3,1.083624,1.403419,1.421812,0.224410,2.679409,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
4,1.046726,1.403419,-0.570187,0.224410,1.517692,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
540,-0.991879,-1.308863,-0.570187,-0.929397,1.517692,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
541,-1.268613,0.047278,-0.570187,-0.929397,-0.805741,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
542,-0.705921,-1.308863,-0.570187,-0.929397,-0.805741,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
543,-1.033389,0.047278,-0.570187,-0.929397,-0.805741,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
target

0      13300000
1      12250000
2      12250000
3      12215000
4      11410000
         ...   
540     1820000
541     1767150
542     1750000
543     1750000
544     1750000
Name: price, Length: 545, dtype: int64

In [12]:
cleaned_data["price"] = target

In [13]:
cleaned_data

,area,bedrooms,bathrooms,stories,parking,x0_yes,x1_yes,x2_yes,x3_yes,x4_yes,x5_yes,x6_semi-furnished,x6_unfurnished,price
0,1.046726,1.403419,1.421812,1.378217,1.517692,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,13300000
1,1.757010,1.403419,5.405809,2.532024,2.679409,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,12250000
2,2.218232,0.047278,1.421812,0.224410,1.517692,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,12250000
3,1.083624,1.403419,1.421812,0.224410,2.679409,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,12215000
4,1.046726,1.403419,-0.570187,0.224410,1.517692,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,11410000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
540,-0.991879,-1.308863,-0.570187,-0.929397,1.517692,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1820000
541,-1.268613,0.047278,-0.570187,-0.929397,-0.805741,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1767150
542,-0.705921,-1.308863,-0.570187,-0.929397,-0.805741,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1750000
543,-1.033389,0.047278,-0.570187,-0.929397,-0.805741,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1750000


In [15]:
cleaned_data.to_csv("dataset/cleanedData.csv",index=False)